In [0]:
#make sure GPU available
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [0]:
# import modules required
from keras.applications.densenet import DenseNet169
from keras.layers import Dense, Flatten
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import numpy
import csv
import imageio
import numpy as np
import cv2
import math
from keras import backend as K
import zipfile
from keras.optimizers import Adam

In [0]:
# upload preprocessed dataset zipfile
from google.colab import files
files.upload()

In [0]:
# extract zip
with zipfile.ZipFile("MURA-preprocessed.zip","r") as zip_ref:
    zip_ref.extractall(".")

In [0]:
# removes filename from a path
def remove_filename(p):
    output = ""
    file_gone = False
    for c in reversed(p):
        if file_gone:
            output += c
        else:
            if c == '/':
                output += c
                file_gone = True
    return output[::-1]

In [0]:
# build model and initialise with imagenet pretrained weights
densenet = DenseNet169(include_top=False, weights='imagenet', input_tensor=None, input_shape=(320,320,3), pooling=None, classes=False)
model = Sequential()
model.add(densenet)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [0]:
# compile model
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [0]:
# gets images, labels and number of normal/abnormal images for a bodypart
def get_training_data(bodypart):
  label_dict = {}
  studies = open("MURA-v1.1/train_labeled_studies.csv", newline='')
  for study in csv.reader(studies):
    label_dict[study[0]] = study[1]
  normal = 0
  abnormal = 0
  paths = open("MURA-v1.1/train_image_paths.csv", newline='')
  images = []
  labels = []
  for path in csv.reader(paths):
    if bodypart.upper() in path[0]:
      images.append(imageio.imread(path[0]))
      label = int(label_dict[remove_filename(path[0])])
      labels.append(label)
      if label == 1:
        abnormal += 1
      else:
        normal += 1
  for i in range(len(images)):
    img = images[i]
    resized_img = cv2.resize(img, (320, 320))
    if len(img.shape) == 2:
        resized_img = np.dstack([resized_img, resized_img, resized_img])
    images[i] = resized_img
  for i in range(len(images)):
    img = images[i]
    images[i] = img[np.newaxis, ...]
  images_tensor = np.concatenate(images, axis=0)
  return (images_tensor, labels, normal, abnormal)

In [0]:
# training function for a particular bodypart
def train_bodypart(bodypart):
  # get data
  training_images, training_labels, normal, abnormal = get_training_data(bodypart)
  normal_fraction = normal / (abnormal + normal)
  abnormal_fraction = abnormal / (abnormal + normal)
  # fit model and evaluate on validation dataset
  model.fit(x=training_images, y=training_labels, class_weight={0: abnormal_fraction, 1: normal_fraction}, epochs=3, batch_size=11)

In [0]:
from google.colab import files
for i in range(3):
  for bodypart in ['elbow', 'forearm', 'hand', 'wrist', 'humerus', 'finger', 'shoulder']:
    train_bodypart(bodypart)
  filename = 'my_model_weights_preprocessed' + str(i) + '.h5'
  model.save_weights(filename)
  files.download(filename)